In [1]:
!sudo apt-get update
!sudo apt-get install --reinstall -y poppler-utils
!apt-get install -y tesseract-ocr
!apt-get install -y libmagic-dev

!pip install -Uq "unstructured[all-docs]" pillow lxml pillow
!pip install -Uq chromadb tiktoken
!pip install -Uq langchain langchain-community langchain-openai langchain-groq
!pip install -Uq python_dotenv
!pip install -Uq unstructured
!pip install -Uq nltk
!pip install -Uq langchain-groq
!pip install -Uq langchain langchain-ollama
!pip install -Uq langchain-huggingface
!pip install -Uq gTTs
!pip install -Uq gradio


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 3s (93.7 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lis

In [2]:
import os
from unstructured.partition.pdf import partition_pdf
import nltk
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import SystemMessage, HumanMessage
from base64 import b64decode
import gradio as gr
from gtts import gTTS
from langdetect import detect, DetectorFactory
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_huggingface import HuggingFaceEmbeddings

# Multi-modal RAG with LangChain

In [3]:

# keys for the services we will use

os.environ["OLLAMA_API_KEY"] = "ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIItDAFT8xNkwkYNtNCL+58HRVaJs6d0NGd6a0dKP1BSt"
os.environ["GROQ_API_KEY"] = "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"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

## Extract the data

Extract the elements of the PDF that we will be able to use in the retrieval process: text, images, tables, etc.

### Partition PDF tables, text, and images

In [4]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

file_path = '/content/directions.pdf'

chunks = partition_pdf(
    filename=file_path,
    infer_table_structure=True,            # extract tables
    strategy="hi_res",                     # mandatory to infer tables
    chunking_strategy="by_title",          # or 'basic'
    max_characters=10000,                  # defaults to 500
    combine_text_under_n_chars=2000,       # defaults to 0
    new_after_n_chars=6000,
)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [5]:
# We get 2 types of elements from the partition_pdf function
set([str(type(el)) for el in chunks])

{"<class 'unstructured.documents.elements.CompositeElement'>"}

In [6]:
# Each CompositeElement containes a bunch of related elements.
# This makes it easy to use these elements together in a RAG pipeline.

chunks[0].metadata.orig_elements

# Separate extracted elements into tables and text

In [7]:
# separate tables from texts
tables = []
texts = []

for chunk in chunks:
    if "Table" in str(type(chunk)):
        tables.append(chunk)

    if "CompositeElement" in str(type((chunk))):
        texts.append(chunk)

## Summarize the data

Create a summary of each element extracted from the PDF. This summary will be vectorized and used in the retrieval process.

### Text and Table summaries

We don't need a multimodal model to generate the summaries of the tables and the text. I will use open source models available on Groq.

In [8]:
# Prompt
prompt_text = """
You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text.

Respond only with the summary, no additionnal comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.

Table or text chunk: {element}

"""
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
model = ChatGroq(temperature=0.5, model="llama-3.1-8b-instant")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [9]:
# Summarize text
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 3})

# Summarize tables
tables_html = [table.metadata.text_as_html for table in tables]
table_summaries = summarize_chain.batch(tables_html, {"max_concurrency": 3})

In [10]:
text_summaries

['Directions around the campus are provided, including walking distances and directions to various locations such as buildings, stairs, elevators, and pedestrian crossings.',
 'The text describes a series of directions to navigate between various buildings, rooms, and elevators within a campus. The directions involve walking, going up or down stairs, and using elevators to reach different locations. The text provides step-by-step instructions for each route, including distances and landmarks to help users navigate the campus.',
 'The document describes walking directions and distances between various locations within a campus, including buildings, entrances, and landmarks such as fountains and a pedestrian cross.',
 'The building layout is described with distances between various entrance/exit points, staircases, elevators, and rooms. Distances range from 2 to 150 meters.',
 'The distances between various locations in the PRECIS building and its surroundings are as follows: \n- 2 meter

## Load data and summaries to vectorstore

### Create the vectorstore

In [13]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="multi_modal_rag", embedding_function=embeddings)

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

### Load the summaries and link the to the original data

In [14]:
# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=summary, metadata={id_key: doc_ids[i]}) for i, summary in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=summary, metadata={id_key: table_ids[i]}) for i, summary in enumerate(table_summaries)
]
retriever.docstore.mset(list(zip(table_ids, tables)))

## RAG pipeline

In [17]:
def parse_docs(docs):
    """Process and separate text data"""
    text = []
    for doc in docs:
        text.append(doc)
    return {"texts": text}


def build_prompt(kwargs):

    docs_by_type = kwargs["context"]
    user_question = kwargs["question"]

    context_text = ""
    if len(docs_by_type["texts"]) > 0:
        for text_element in docs_by_type["texts"]:
            context_text += text_element.text

    # construct prompt with context (including images)
    prompt_template = f"""
    You are an AI assistant specialized in finding the best possible route between a destination and another.
    Your task is to analyze a short a pdf that contains direction advices on how different locations connect to each other.

    Specifically, provide the following details:
      1. Offer the shortest path according to directions you have.
      2. Use as few intermediate checkpoints as you can.
      3. A classroom name is composed of two uppercased letters and three digits (example given: EC105).
      4. If the user asks for a classroom you do not have information about, give an aproximate path. For example: the users asks for EC103, an aproximate path should be leading to EC105, because they are in the same building.
      5. You will calculate the shortesc path in meters, not according to the number of intermediate locations.
      6. If locatia A is connected to location B by a number of meters, then also location B is connected to location A by a number of meters.
      7. When you give direction, you should specify when you turn left and right.
      8. Make you do not repeat the yourself.
      9. You will prompt only one possible route. The best one.
      10. You do not need to specify the total distance.

    Context: {context_text}
    Question: {user_question}
    """

    prompt_content = [{"type": "text", "text": prompt_template}]


    return ChatPromptTemplate.from_messages(
        [
            HumanMessage(content=prompt_content),
        ]
    )


chain = (
    {
        "context": retriever | RunnableLambda(parse_docs),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt)
    | ChatGroq(temperature=0.5, model="llama-3.2-11b-vision-preview")
    | StrOutputParser()
)

chain_with_sources = {
    "context": retriever | RunnableLambda(parse_docs),
    "question": RunnablePassthrough(),
} | RunnablePassthrough().assign(
    response=(
        RunnableLambda(build_prompt)
        | ChatGroq(temperature=0.5, model="llama-3.2-11b-vision-preview")
        | StrOutputParser()
    )
)


In [18]:
response = chain.invoke(
    "how do i get from EC105 to EC Caffeteria?"
)

print(response)

To get from EC105 to EC Caffeteria, follow these directions:

From EC105, go to the left to find the stairs (EC Stairs (2)). Go up the stairs to the first floor. Then, turn to the right twice. If you go forward, through the glass doors, you will find the cafeteria.


In [19]:
# Set a seed to ensure consistent language detection
DetectorFactory.seed = 0

def chatbot(user_input):
    response_text = chain.invoke(user_input)

    # Detect the language of the response_text
    detected_lang = detect(response_text)

    # Create the audio file from the response
    tts = gTTS(text=response_text, lang=detected_lang)
    audio_file = "response.mp3"
    tts.save(audio_file)

    return response_text, audio_file

# Define the Gradio interface
interface = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs=["text", "audio"],
    title="Milly guides you",
    description="Type something and the chatbot will respond with text and audio!",
)

# Launch the interface
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e7ed758da00f45c9a3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
